In [1]:
import numpy as np 
import pandas as pd
import pandas.util.testing as tm
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import calendar
import requests as re
import time
import os
from pprint import pprint
import time
from scipy.stats import f_oneway



plt.style.use('seaborn')
sns.set_style("darkgrid", {"axes.facecolor": ".9"})

%matplotlib inline

C:\Users\Jesse\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  This is separate from the ipykernel package so we can avoid doing imports until


#### **Task 3**: Does the weather affect the delay? 
Use the API to pull the weather information for flights. There is no need to get weather for ALL flights. We can choose the right representative sample. Let's focus on four weather types:
- sunny
- cloudy
- rainy
- snow.
Test the hypothesis that these 4 delays are from the same distribution. If they are not, which ones are significantly different?


Using dark sky: https://darksky.net/

In [31]:
api_key='80c6c754465a0a3a359356dbb9be865c'

In [3]:
airports = pd.read_csv("C:/Users/Jesse/Downloads/datasets_810_1496_airports.csv")
airports.shape

(322, 7)

In [32]:
data_path = 'C:/Users/Jesse/Desktop/midterm_data/data/'
flights_df = pd.read_csv(data_path + 'flights.csv')

C:\Users\Jesse\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://www.qualtrics.com/m/assets/wp-content/uploads/2020/04/Sample-size-formula-graphic-1.png", 
     width=700)

https://www.qualtrics.com/experience-management/research/determine-sample-size/


A low standard deviation means that all the values will be clustered around the mean number, whereas a high standard deviation means they are spread out across a much wider range with very small and very large outlying figures. Since you haven’t yet run your survey, a safe choice is a standard deviation of .5 which will help make sure your sample size is large enough.

In [33]:
zscore = 1.96
stddev = 0.5
error = 0.05

def get_sample_size(zscore, stddev, error):
    return ((zscore**2)*stddev*(1-stddev))/(error**2)
    
sample_size = get_sample_size(zscore, stddev, error)
## 385 samples size keeps 95% interval and 5% standard error

sample_size

384.1599999999999

In [34]:
flights_df['fl_date'] = pd.to_datetime(flights_df['fl_date'], format ='%Y-%m-%d')

In [35]:
sample=flights_df.loc[(flights_df['fl_date'].dt.month.isin([12,1,2]))]


In [36]:
# Get all the unique cities
cities=sample['origin_city_name'].unique()

# Create new column just for cityname
sample['city'] = sample['origin_city_name'].apply(lambda x: x.split(', ')[0])


# Create new dataframe with city, lat and lon
x = airports.loc[airports['CITY'].isin(sample['city']), ['CITY', 'LATITUDE', 'LONGITUDE']]
x.rename({'CITY': 'city'},axis=1, inplace=True)

C:\Users\Jesse\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [37]:
snowiest_cities = ['Rochester', 'Lakewood', 'Anchorage', 'Syracuse', 'Fort Collins', 'Buffalo', 'Green Bay', 'Erie', 'Grand Rapids', ]


sample = sample.loc[sample['city'].isin(snowiest_cities)].sample(500)

In [38]:
df = pd.merge(right=x, left=sample, on='city')

In [39]:
df['fl_date'] = (df['fl_date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')  

In [40]:
df.shape

(593, 45)

In [41]:
def get_weather(row):
    time.sleep(3)
    response = re.get(f'https://api.darksky.net/forecast/{api_key}/{row["LATITUDE"]},{row["LONGITUDE"]},{row["fl_date"]}').json()
    try:
        return response['currently']['icon']       
    except:
        return 'cloudy'

In [42]:
df['weather'] = df.apply(lambda row:get_weather(row),axis=1)

In [43]:
df['weather'].value_counts()

cloudy                 269
clear-night            130
partly-cloudy-night     77
snow                    47
rain                    25
fog                     16
partly-cloudy-day       15
clear-day               10
wind                     4
Name: weather, dtype: int64

In [56]:
# # df.to_csv('./flights_weather.csv')

# df = pd.read_csv('./flights_weather.csv')

In [57]:
def change_weather(weather):
    if weather in ['partly-cloudy-day', 'clear-day']:
        return 'sunny'
    else:
        return weather
    
df['weather'] = df['weather'].apply(lambda x: change_weather(x))

In [59]:
indexdrop = df.loc[df['weather'].isin(['clear-night', 'partly-cloudy-night', 'fog', 'wind'])].index

df.drop(indexdrop, axis=0, inplace=True)

In [60]:
df['weather'].value_counts()

cloudy    269
snow       47
sunny      25
rain       25
Name: weather, dtype: int64

In [76]:
df.to_csv('./flights_weather_cleaned.csv')

In [119]:
df = pd.read_csv('./flights_weather_cleaned.csv')

In [120]:
cloudy = df.loc[df['weather'] == 'cloudy', 'dep_delay'].dropna().values
snow = df.loc[df['weather'] == 'snow', 'dep_delay'].dropna().values
sunny = df.loc[df['weather'] == 'sunny', 'dep_delay'].dropna().values
rain = df.loc[df['weather'] == 'rain', 'dep_delay'].dropna().values

In [121]:
F, p = f_oneway(cloudy, snow, sunny, rain)

F, p

(0.7599279173334405, 0.5172053097339369)

In [122]:
# Getting rid of outliers and retesting

In [123]:
def reject_outliers(data, m=3):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

In [124]:
cloudy = reject_outliers(cloudy)
snow = reject_outliers(snow)
sunny = reject_outliers(sunny)
rain = reject_outliers(rain)



F, p = f_oneway(cloudy, snow, sunny, rain)

print("p-value",p)
if p <0.05:
  print("we reject null hypothesis")
else:
  print("we accept null hypothesis")

p-value 0.07529835703573715
we accept null hypothesis
